In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mplt
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')
submission_file = pd.read_csv('Data/sample_submission.csv')

In [4]:
train_data.drop(columns=['id'], axis=1, inplace=True)

In [5]:
print(train_data.shape)
print(test_data.shape)

(600000, 101)
(540000, 101)


In [6]:
X, y = train_data.drop(columns = ['target']), train_data['target']


In [7]:
dtypes = train_data.dtypes
dtypes = dtypes[dtypes != 'object']
features = list(set(dtypes.index) - set(['target']))

len(features)

100

In [8]:
train_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,...,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,...,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,...,4.88300,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.14112,0.124464,0.148209,0
3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,...,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,...,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [9]:
X.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
count,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,...,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000
mean,0.306508,2.497590,306.644536,2.647901,0.177850,2.556832,2.699650,2.571593,2.538273,0.134370,...,0.071252,2.444471,0.155260,0.059407,0.144932,0.106419,2.547853,2.590159,0.158881,0.123048
std,0.522450,1.554018,551.743893,1.544529,0.417488,1.562527,1.564000,1.549361,1.532988,0.421892,...,0.112654,1.542509,0.548397,0.119426,0.462015,0.209128,1.558427,1.525091,0.436190,0.264896
min,-3.797450,-1.223960,-1842.530000,-1.368560,-3.206210,-1.169770,-1.059310,-1.281970,-1.242020,-2.577840,...,-3.676990,-1.217700,-9.761770,-4.666240,-3.101500,-1.276540,-1.584740,-1.254730,-3.993500,-2.783380
25%,0.026222,1.186237,43.573400,1.442028,0.019709,1.261038,1.385820,1.333848,1.292163,0.019563,...,0.020496,1.214177,0.018904,0.024483,0.017055,0.025461,1.247888,1.348078,0.013536,0.018105
50%,0.097788,2.516500,133.626000,2.634130,0.061586,2.590425,2.801255,2.557985,2.475880,0.058752,...,0.054546,2.386845,0.068906,0.056649,0.063439,0.062151,2.601940,2.682090,0.058058,0.058471
75%,0.397184,3.787630,302.262250,3.907640,0.112712,3.813662,3.996913,3.823450,3.804360,0.101046,...,0.091619,3.693872,0.125165,0.088162,0.113114,0.102016,3.820665,3.839520,0.110718,0.104872
max,8.781500,6.226720,6119.280000,6.521150,8.265470,6.515070,6.586780,6.258770,6.389670,7.078460,...,6.482940,6.573890,18.412800,10.211800,8.623270,3.657220,6.254360,6.145300,10.767000,5.988110


In [10]:
y.describe()

count    600000.000000
mean          0.506010
std           0.499964
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: target, dtype: float64

In [11]:
X = X.astype(float)
y = y.astype(int)

X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

print(f'check for null value in X: {X.isnull().sum().sum()}')
print(f'check for null value in y: {y.isnull().sum().sum()}')

check for null value in X: 0
check for null value in y: 0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [13]:
scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)
X_test = pd.DataFrame(scaler.fit_transform(X_test[num_cols]), columns=num_cols)

## Initial Baseline Implementation using KerasClassifier

In [83]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):

	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=100, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(64, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(32, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(16, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))	
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

	# create model
	#model = Sequential()
	#model.add(Dense(32, input_dim=100, kernel_initializer=init, activation='relu'))
	#model.add(Dense(16, kernel_initializer=init, activation='relu'))
	#model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))	
	
	# create model
	#model = Sequential()
	#model.add(Dense(128, input_dim=100, kernel_initializer=init, activation='relu'))
	#model.add(Dense(64, kernel_initializer=init, activation='relu'))
	#model.add(Dense(32, kernel_initializer=init, activation='relu'))
	#model.add(Dense(16, kernel_initializer=init, activation='relu'))
	#model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
#optimizers = ['rmsprop', 'adam']
#init = ['glorot_uniform', 'normal', 'uniform']
#epochs = [100, 150]
#batches = [1024, 2048]

optimizers = ['adam']
init = ['glorot_uniform']
epochs = [1000] #[100]
batches = [1024, 2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

y_hat_train = grid.predict(X_train)
y_hat_test = grid.predict(X_test)

train_score = accuracy_score(y_train, y_hat_train, normalize=False)
print(f'trian score: {train_score / y_train.shape[0]}')

test_score = accuracy_score(y_test, y_hat_test, normalize=False)
print(f'test score: {test_score / y_test.shape[0]}')

precision_train_logit_grid = precision_score(y_train, y_hat_train) * 100
precision_test_logit_grid = precision_score(y_test, y_hat_test) * 100

recall_train_logit_grid = recall_score(y_train, y_hat_train) * 100
recall_test_logit_grid = recall_score(y_test, y_hat_test) * 100

f1_score_train_logit=f1_score(y_train, y_hat_train) * 100
f1_score_test_logit=f1_score(y_test, y_hat_test) * 100

auc_score_train_logit_grid = roc_auc_score(y_train, y_hat_train) * 100
auc_score_test_logit_grid = roc_auc_score(y_test, y_hat_test) * 100

print("Precision = {:.2f}% , recall = {:.2f}% and f1_score={:.2f}% of the Logistic Regression Model on the training data.".format(precision_train_logit_grid, recall_train_logit_grid, f1_score_train_logit))
print("Precision = {:.2f}% , recall = {:.2f}% and f1_score={:.2f}% of the Logistic Regression Model on the validation data.".format(precision_test_logit_grid, recall_test_logit_grid, f1_score_test_logit_grid))
print("ROC_AUC Score = {:.2f}%  of Logistic Regression Model on the training data.".format(auc_score_train_logit_grid))
print("ROC_AUC Score = {:.2f}%  of Logistic Regression Model on the validation data.".format(auc_score_test_logit_grid))

# public score: 0.74736 (427th)

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_74479/4223488330.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, verbose=0)
2021-11-25 16:08:54.519827: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 16:08:54.519827: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 16:08:54.520015: I tensorflow/core/platform/cpu_f

Best: 0.740409 using {'batch_size': 1024, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.740409 (0.001741) with: {'batch_size': 1024, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.739987 (0.002545) with: {'batch_size': 2048, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
trian score: 0.7581266666666666
test score: 0.7430866666666667
Precision = 75.69% , recall = 76.89% and f1_score=76.28% of the Logistic Regression Model on the training data.
Precision = 74.26% , recall = 75.39% and f1_score=75.00% of the Logistic Regression Model on the validation data.
ROC_AUC Score = 75.80%  of Logistic Regression Model on the training data.
ROC_AUC Score = 74.29%  of Logistic Regression Model on the validation data.


## Define Common Functions

In [17]:
def create_keras_sequential_model(optimizer='adam', init='glorot_uniform'):

	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=100, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(64, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(32, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(16, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))	
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

	return model

In [22]:
def report_results(model_name, y_test, y_train, grid_search_model, grid_search_results):

	# summarize results
	print("Best: %f using %s" % (grid_search_results.best_score_, grid_search_results.best_params_))
	means = grid_search_results.cv_results_['mean_test_score']
	stds = grid_search_results.cv_results_['std_test_score']
	params = grid_search_results.cv_results_['params']

	for mean, stdev, param in zip(means, stds, params):
		print("%f (%f) with: %r" % (mean, stdev, param))

	y_hat_train = grid_search_model.predict(X_train)
	y_hat_test = grid_search_model.predict(X_test)

	train_score = accuracy_score(y_train, y_hat_train, normalize=False)
	print(f'trian score: {train_score / y_train.shape[0]}')

	test_score = accuracy_score(y_test, y_hat_test, normalize=False)
	print(f'test score: {test_score / y_test.shape[0]}')

	precision_train_score = precision_score(y_train, y_hat_train) * 100
	precision_test_score = precision_score(y_test, y_hat_test) * 100

	recall_train_score = recall_score(y_train, y_hat_train) * 100
	recall_test_score = recall_score(y_test, y_hat_test) * 100

	f1_train_score = f1_score(y_train, y_hat_train) * 100
	f1_test_score = f1_score(y_test, y_hat_test) * 100

	auc_train_score = roc_auc_score(y_train, y_hat_train) * 100
	auc_test_score = roc_auc_score(y_test, y_hat_test) * 100

	print("Precision = {:.2f}% , recall = {:.2f}% and f1_score={:.2f}% of the % model on the training data.".format(precision_train_score, recall_train_score, f1_train_score, model_name))
	print("Precision = {:.2f}% , recall = {:.2f}% and f1_score={:.2f}% of the % model on the validation data.".format(precision_test_score, recall_test_score, f1_test_score, model_name))
	print("ROC_AUC Score = {:.2f}%  of the % model on the training data.".format(auc_train_score, model_name))
	print("ROC_AUC Score = {:.2f}%  of the % model on the validation data.".format(auc_test_score, model_name))


## Tune Batch Size and Number of Eposchs

In [18]:
# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

# grid search epochs, batch size and optimizer
#optimizers = ['rmsprop', 'adam']
#init = ['glorot_uniform', 'normal', 'uniform']
#epochs = [100, 150]
#batches = [1024, 2048]

optimizers = ['adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [100, 200, 300]
batches = [1024, 2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)


/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_847/4096780091.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 16:19:31.204977: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 16:19:31.205013: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 16:19:31.205020: I tensorflow/core/

In [23]:
report_results('KerasClassifier', y_test, y_train, grid, grid_result)


Best: 0.742073 using {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.741736 (0.002229) with: {'batch_size': 1024, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.740542 (0.002268) with: {'batch_size': 1024, 'epochs': 100, 'init': 'normal', 'optimizer': 'adam'}
0.740862 (0.002184) with: {'batch_size': 1024, 'epochs': 100, 'init': 'uniform', 'optimizer': 'adam'}
0.741229 (0.002358) with: {'batch_size': 1024, 'epochs': 200, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.740569 (0.002564) with: {'batch_size': 1024, 'epochs': 200, 'init': 'normal', 'optimizer': 'adam'}
0.740144 (0.001736) with: {'batch_size': 1024, 'epochs': 200, 'init': 'uniform', 'optimizer': 'adam'}
0.740784 (0.002146) with: {'batch_size': 1024, 'epochs': 300, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.740707 (0.001958) with: {'batch_size': 1024, 'epochs': 300, 'init': 'normal', 'optimizer': 'adam'}
0.740747 (0.001736) with: {'batch_size': 1024, 'epochs': 300, 

In [24]:
# re-train with best parameter and full set of train dataset

X_train, y_train = X, y

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)

# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

optimizers = ['adam']
init = ['glorot_uniform']
epochs = [100]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# public score: 0.74736 (427th)

# public score: 0.74591 (second run)

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_847/219623009.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 19:07:34.831019: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 19:07:34.831102: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 19:07:34.833789: I tensorflow/core/

In [25]:
report_results('KerasClassifier', y_test, y_train, grid, grid_result)

Best: 0.743600 using {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.743600 (0.004309) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
trian score: 0.7510366666666667
test score: 0.75208
Precision = 75.08% , recall = 76.03% and f1_score=75.55% of the % model on the training data.
Precision = 75.22% , recall = 76.11% and f1_score=75.66% of the % model on the validation data.
ROC_AUC Score = 75.09%  of the % model on the training data.
ROC_AUC Score = 75.20%  of the % model on the validation data.


## Round 2 - Number of Eposchs

In [26]:
# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['adam']
init = ['glorot_uniform']
epochs = [1000]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

report_results('KerasClassifier', y_test, y_train, grid, grid_result)

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_847/2622259205.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 19:20:43.306463: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 19:20:43.315967: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 19:20:43.316200: I tensorflow/core/

Best: 0.742638 using {'batch_size': 2048, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.742638 (0.004250) with: {'batch_size': 2048, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
trian score: 0.75463
test score: 0.7556133333333334
Precision = 75.39% , recall = 76.47% and f1_score=75.93% of the % model on the training data.
Precision = 75.52% , recall = 76.54% and f1_score=76.03% of the % model on the validation data.
ROC_AUC Score = 75.45%  of the % model on the training data.
ROC_AUC Score = 75.55%  of the % model on the validation data.


In [27]:
# re-train with best parameter and full set of train dataset

X_train, y_train = X, y

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)

# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

optimizers = ['adam']
init = ['glorot_uniform']
epochs = [1000]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# public score: ?

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_847/3139461891.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 20:56:13.537228: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 20:56:13.539045: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 20:56:13.539699: I tensorflow/core

In [28]:
report_results('KerasClassifier', y_test, y_train, grid, grid_result)


Best: 0.743443 using {'batch_size': 2048, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.743443 (0.004062) with: {'batch_size': 2048, 'epochs': 1000, 'init': 'glorot_uniform', 'optimizer': 'adam'}
trian score: 0.75469
test score: 0.7556133333333334
Precision = 75.45% , recall = 76.37% and f1_score=75.91% of the % model on the training data.
Precision = 75.56% , recall = 76.47% and f1_score=76.01% of the % model on the validation data.
ROC_AUC Score = 75.46%  of the % model on the training data.
ROC_AUC Score = 75.55%  of the % model on the validation data.


## Tune the Training Optimization Algorithm

In [51]:
# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

#optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizers = ['Adamax']
init = ['glorot_uniform']
epochs = [100]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)


/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/467874160.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-25 23:04:18.145394: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 23:04:18.248205: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
report_results('KerasClassifier', y_test, y_hat_test, y_train, y_hat_train, grid, grid_result)

Best: 0.744527 using {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'Adamax'}
0.744527 (0.003838) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'Adamax'}
trian score: 0.7519483333333333
test score: 0.7527933333333333
Precision = 75.20% , recall = 76.06% and f1_score=75.63% of the % model on the training data.
Precision = 75.32% , recall = 76.12% and f1_score=75.72% of the % model on the validation data.
ROC_AUC Score = 75.18%  of the % model on the training data.
ROC_AUC Score = 75.27%  of the % model on the validation data.


In [53]:
# re-train with best parameter

X_train, y_train = X, y

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)

# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

optimizers = ['Adamax']
init = ['glorot_uniform']
epochs = [100]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# public score: 0.74599

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/3837577376.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-25 23:19:55.142051: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 23:19:55.142488: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 23:19:55.153509: I tensorflow/co

## Tune Network Weight Initialization

In [61]:
# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

#optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizers = ['Adamax']
init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
epochs = [100]
batches = [2048]
learn_rate = [0.001, 0.01]
momentum = [0.0, 0.2, 0.4]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(X_train, y_train)
grid_result = grid.fit(X_train, y_train)


/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/2345569135.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
/Users/kyle/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
2021-11-26 00:23:42.312301: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 00:23:42.316979: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with on

In [62]:
report_results('KerasClassifier', y_test, y_hat_test, y_train, y_hat_train, grid, grid_result)

Best: 0.744352 using {'batch_size': 2048, 'epochs': 100, 'init': 'he_uniform', 'optimizer': 'Adamax'}
0.742848 (0.004295) with: {'batch_size': 2048, 'epochs': 100, 'init': 'uniform', 'optimizer': 'Adamax'}
0.743383 (0.003994) with: {'batch_size': 2048, 'epochs': 100, 'init': 'lecun_uniform', 'optimizer': 'Adamax'}
0.743197 (0.004248) with: {'batch_size': 2048, 'epochs': 100, 'init': 'normal', 'optimizer': 'Adamax'}
0.460970 (0.027027) with: {'batch_size': 2048, 'epochs': 100, 'init': 'zero', 'optimizer': 'Adamax'}
0.743533 (0.004100) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_normal', 'optimizer': 'Adamax'}
0.743728 (0.004226) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'Adamax'}
0.744147 (0.004041) with: {'batch_size': 2048, 'epochs': 100, 'init': 'he_normal', 'optimizer': 'Adamax'}
0.744352 (0.004369) with: {'batch_size': 2048, 'epochs': 100, 'init': 'he_uniform', 'optimizer': 'Adamax'}
trian score: 0.75103
test score: 0.75176
Preci

In [63]:
# re-train with best parameter

X_train, y_train = X, y

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)

# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

optimizers = ['Adamax']
init = ['he_uniform']
epochs = [100]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# public score: 0.74599

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/184325068.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 07:46:38.863666: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 07:46:38.863796: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 07:46:38.863824: I tensorflow/cor

## Tune Learning Rate

In [71]:
def create_learning_rate_model(optimizer='adam', init='glorot_uniform', learn_rate=0.001):

	model = Sequential()
	model.add(Dense(128, input_dim=100, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(64, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(32, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(16, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))	
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

	model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learn_rate), metrics=['accuracy'])

	return model

# create model
model = KerasClassifier(build_fn=create_learning_rate_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['adam']
init = ['glorot_uniform']
epochs = [100]
batches = [2048]
# Learning rate controls how much to update the weight at the end of each batch 
learn_rate = [0.0001, 0.001, 0.01]
# and the momentum controls how much to let the previous update influence the current weight update
momentum = [0.0, 0.2, 0.4] # ingnore for now

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init, learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

report_results('KerasClassifier', y_test, y_hat_test, y_train, y_hat_train, grid, grid_result)


/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/3938506379.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_learning_rate_model, verbose=0)
2021-11-26 08:22:11.361209: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 08:22:11.361474: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 08:22:11.363451: I tensorflow/core/

Best: 0.744832 using {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'learn_rate': 0.0001, 'optimizer': 'adam'}
0.744832 (0.004127) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'learn_rate': 0.0001, 'optimizer': 'adam'}
0.743538 (0.003970) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'learn_rate': 0.001, 'optimizer': 'adam'}
0.732608 (0.004847) with: {'batch_size': 2048, 'epochs': 100, 'init': 'glorot_uniform', 'learn_rate': 0.01, 'optimizer': 'adam'}
trian score: 0.7494466666666667
test score: 0.7503733333333333
Precision = 74.93% , recall = 75.88% and f1_score=75.40% of the % model on the training data.
Precision = 75.06% , recall = 75.93% and f1_score=75.49% of the % model on the validation data.
ROC_AUC Score = 74.93%  of the % model on the training data.
ROC_AUC Score = 75.03%  of the % model on the validation data.


In [72]:
# re-train with best parameter

def create_learning_rate_model(optimizer='adam', init='glorot_uniform', learn_rate=0.0001):

	model = Sequential()
	model.add(Dense(128, input_dim=100, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(64, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(32, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(16, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.2))	
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

	model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learn_rate), metrics=['accuracy'])

	return model

X_train, y_train = X, y

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

scaler = StandardScaler()

num_cols = X_train.select_dtypes(['integer', 'float']).columns

X_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)

# create model
model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)

optimizers = ['adam']
init = ['glorot_uniform']
epochs = [100]
batches = [2048]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# public score: 0.74591

/var/folders/v3/ph9z3jvd3h32sv82kzzmklym0000gn/T/ipykernel_76605/3824939399.py:32: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_keras_sequential_model, verbose=0)
2021-11-26 09:16:22.577960: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 09:16:22.578015: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 09:16:22.578014: I tensorflow/co

In [29]:
data_test_norm = pd.DataFrame(scaler.transform(test_data[num_cols]), columns = num_cols)

test_predict = grid.predict_proba(data_test_norm)[::,1]
test_predict = test_predict.astype(float)
array = np.array(test_predict).tolist()
df = pd.DataFrame(test_data['id'])
df['id'] = df['id'].astype(int)
df['target'] = np.array(array)
df.to_csv('results/keras_results.csv', sep=',', encoding='utf-8', index=False)

In [30]:
%cd results/

!kaggle  competitions  submit -c tabular-playground-series-nov-2021 -f keras_results.csv -m "keras implementation"

%cd ..

/Users/kyle/Documents/github-data-research-team/kaggle-competitions/tabular-playground-series-nov-2021/results
100%|███████████████████████████████████████| 13.6M/13.6M [00:21<00:00, 674kB/s]
Successfully submitted to Tabular Playground Series - Nov 2021/Users/kyle/Documents/github-data-research-team/kaggle-competitions/tabular-playground-series-nov-2021


In [31]:
!kaggle competitions submissions -c tabular-playground-series-nov-2021 -q

fileName           date                 description                         status    publicScore  privateScore  
-----------------  -------------------  ----------------------------------  --------  -----------  ------------  
keras_results.csv  2021-11-27 06:36:55  keras implementation                complete  0.74616      None          
keras_results.csv  2021-11-27 00:15:06  keras implementation                complete  0.74591      None          
keras_results.csv  2021-11-26 06:57:18  keras implementation                complete  0.74599      None          
keras_results.csv  2021-11-26 05:34:15  keras implementation                complete  0.74736      None          
keras_results.csv  2021-11-26 05:08:25  keras implementation                complete  0.73143      None          
keras_results.csv  2021-11-26 02:27:10  KerasClassfier baseline submission  complete  0.73143      None          
keras_results.csv  2021-11-26 00:05:33  KerasClassfier baseline submission  complete  0.

## 